# Fase 1: Unión de Conjuntos de Datos

### 1. Lectura y Exploración Inicial

In [139]:
import pandas as pd
import numpy as np
import json
import os 
import re

import sys 
sys.path.append("../")
from src import soporte_limpieza

In [140]:
os.chdir("../datos")
datos = os.listdir()[:9]
datos

['datos-2013.csv',
 'datos-2014.csv',
 'datos-2015.csv',
 'datos-2016.csv',
 'datos-2017.csv',
 'datos-2018.csv',
 'datos-2019.csv',
 'datos-2020.csv',
 'datos-2021.csv']

Creamos una lista con los df

In [141]:
dic_df ={}
lista_anios = []

for i in datos:
    df = pd.read_csv(f"../datos/{i}", sep=";", parse_dates=["DATA LANÇAMENTO"])
    # print(i)
    año = re.findall("\d{4}", i)[0]
    dic_df[int(año)] = df
    lista_anios.append(int(año))


C:\Users\Elena\AppData\Local\Temp\ipykernel_16972\875906004.py:5: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df = pd.read_csv(f"../datos/{i}", sep=";", parse_dates=["DATA LANÇAMENTO"])
C:\Users\Elena\AppData\Local\Temp\ipykernel_16972\875906004.py:5: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df = pd.read_csv(f"../datos/{i}", sep=";", parse_dates=["DATA LANÇAMENTO"])
C:\Users\Elena\AppData\Local\Temp\ipykernel_16972\875906004.py:5: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df = pd.read_csv(f"../datos/{i}", sep=";", parse_dates=["DATA LANÇAMENTO"])
C:\Users\Elena\AppData\Local\Temp\ipykernel_16972\875906004.py:5: UserWarning: Parsin

In [142]:
# Observamos el formato de los dataframes
for elem in lista_anios:
    print(dic_df[elem].shape)

(4498, 16)
(4553, 16)
(4523, 16)
(194533, 16)
(190479, 16)
(173944, 16)
(176828, 16)
(142348, 16)
(134593, 16)


Se observa que el volumen de datos es mucho menor en los 3 primeros años, pero todos tienen el mismo número de columnas. Vamos a cambiar el nombre de las columnas de todos los data frames para facilitar la comprensión.

In [143]:
cambio_nombre_columnas = {"CÓDIGO ÓRGÃO SUPERIOR":"id organizacion superior",
                          "NOME ÓRGÃO SUPERIOR":"organizacion superior",
                          "CÓDIGO ÓRGÃO":"id organizacion",
                          "NOME ÓRGÃO":"organizacion",
                          "CÓDIGO UNIDADE GESTORA":"id unidad gestora",
                          "NOME UNIDADE GESTORA":"unidad gestora",
                          "CATEGORIA ECONÔMICA":"categoria economica",
                          "ORIGEM RECEITA":"origen ingreso",
                          "ESPÉCIE RECEITA":"tipo ingreso",
                          "DETALHAMENTO":"detalles",
                          "VALOR PREVISTO ATUALIZADO":"valor previsto", #Este valor era el previsto que se iba a recaudar
                          "VALOR LANÇADO":"valor registrado", #Este valor es el que han apuntado oficialmente como recaudado 
                          "VALOR REALIZADO":"valor recaudado", #Este valor es el que realmente han recaudado
                          "PERCENTUAL REALIZADO":"porcentaje recaudado", #Porcetanje del valor recaudado respecto al previsto
                          "DATA LANÇAMENTO":"fecha recaudacion",
                          "ANO EXERCÍCIO":"anio recaudacion"
                          }

In [144]:
for elem in lista_anios:
    dic_df[elem].rename(columns=cambio_nombre_columnas, inplace=True)

In [145]:
dic_df[2019].head(2)

,id organizacion superior,organizacion superior,id organizacion,organizacion,id unidad gestora,unidad gestora,categoria economica,origen ingreso,tipo ingreso,detalles,valor previsto,valor registrado,valor recaudado,porcentaje recaudado,fecha recaudacion,anio recaudacion
0,63000.0,Advocacia-Geral da União,63000.0,NaN,110060.0,COORD. GERAL DE ORC. FIN. E ANAL. CONT. - AGU,Receitas Correntes,Receita de Serviços,Serviços Administrativos e Comerciais Gerais,INSCR.EM CONCURSOS E PROC.SELETIVOS-PRINCIPAL,"0,00","0,00","-95,00","0,00",12/06/2019,2019.0
1,63000.0,NaN,63000.0,Advocacia-Geral da União - Unidades com víncul...,110060.0,COORD. GERAL DE ORC. FIN. E ANAL. CONT. - AGU,Receitas Correntes,Receita de Serviços,Serviços Administrativos e Comerciais Gerais,INSCR.EM CONCURSOS E PROC.SELETIVOS-PRINCIPAL,"0,00","0,00","-380,00","0,00",07/05/2019,NaN


No tenía muy claro si el año de la columna fecha recaudacion coincide con el de la columna anio recaudacion, en gan parte de los casos era asi y he querido ver donde no coincidian descubriendo que hay una gran cantidad de valores en Nan, pensé en asignar al año de recaudacion el año de la columna fecha recaudacion pero hay filas donde ambos son Nan, por lo que no es una buena estrategia. Como hemos vito que solo hay un valor único que es el año que coincide co el nombre del csv, podemos sustituir todos los Nan del año de recaudación por el año del csv. Los Nan de la fecha de recaudación los dejamos sin tratar y si fuera necesario en el futuro ya veremos que hacemos con ellos.

In [146]:
dic_df[2013].shape

(4498, 16)

In [147]:
dic_df[2013].dtypes

id organizacion superior           float64
organizacion superior               object
id organizacion                    float64
organizacion                        object
id unidad gestora                  float64
unidad gestora                      object
categoria economica                 object
origen ingreso                      object
tipo ingreso                        object
detalles                            object
valor previsto                      object
valor registrado                    object
valor recaudado                     object
porcentaje recaudado                object
fecha recaudacion           datetime64[ns]
anio recaudacion                   float64
dtype: object

In [148]:
filtro = pd.to_datetime(dic_df[2013]["fecha recaudacion"]).dt.year != dic_df[2013]["anio recaudacion"]
df_filtro = dic_df[2013][filtro]
df_filtro.head(1)

,id organizacion superior,organizacion superior,id organizacion,organizacion,id unidad gestora,unidad gestora,categoria economica,origen ingreso,tipo ingreso,detalles,valor previsto,valor registrado,valor recaudado,porcentaje recaudado,fecha recaudacion,anio recaudacion
0,63000.0,NaN,63000.0,Advocacia-Geral da União - Unidades com víncul...,110060.0,COORD. GERAL DE ORC. FIN. E ANAL. CONT. - AGU,Receitas Correntes,Outras Receitas Correntes,"Bens, Direitos e Valores Incorporados ao Patr",REC.DIVIDA ATIVA NAO TRIBUTARIA DE OUTRAS REC,"0,00","0,00","1297,13","0,00",2013-12-31,NaN


In [149]:
dic_df[2013][dic_df[2013]["anio recaudacion"].isnull() & dic_df[2013]["fecha recaudacion"].isnull()].head(1)

,id organizacion superior,organizacion superior,id organizacion,organizacion,id unidad gestora,unidad gestora,categoria economica,origen ingreso,tipo ingreso,detalles,valor previsto,valor registrado,valor recaudado,porcentaje recaudado,fecha recaudacion,anio recaudacion
115,22000.0,NaN,22905.0,Fundo de Defesa da Economia Cafeeira,130137.0,NaN,Receitas Correntes,Outras Receitas Correntes,"Indenizações, restituições e ressarcimentos",RESTITUICOES DE CONVENIOS,"0,00","0,00","40472,45","0,00",NaT,NaN


In [150]:
for elem in lista_anios:
    print(dic_df[elem]["anio recaudacion"].unique())

[  nan 2013.]
[2014.   nan]
[2015.   nan]
[2016.   nan]
[2017.   nan]
[  nan 2018.]
[2019.   nan]
[2020.   nan]
[2021.   nan]


In [151]:
for anio in lista_anios:
    dic_df[anio]["anio recaudacion"]=dic_df[anio]["anio recaudacion"].fillna(anio)

In [152]:
for elem in lista_anios:
  print(dic_df[elem]["anio recaudacion"].unique())

[2013.]
[2014.]
[2015.]
[2016.]
[2017.]
[2018.]
[2019.]
[2020.]
[2021.]


---------------------------

Se observó antes que los tipos de columnas numércicas como son, valor previsto, registrado y recaudado son de tipo object cuano deberían ser un float,cambiaremos los valores a float.

In [153]:
for anio in lista_anios:
    print(f"\n{anio}")
    print(dic_df[anio][["valor previsto", "valor registrado", "valor recaudado", "porcentaje recaudado"]].dtypes)


2013
valor previsto          object
valor registrado        object
valor recaudado         object
porcentaje recaudado    object
dtype: object

2014
valor previsto          object
valor registrado        object
valor recaudado         object
porcentaje recaudado    object
dtype: object

2015
valor previsto          object
valor registrado        object
valor recaudado         object
porcentaje recaudado    object
dtype: object

2016
valor previsto          object
valor registrado        object
valor recaudado         object
porcentaje recaudado    object
dtype: object

2017
valor previsto          object
valor registrado        object
valor recaudado         object
porcentaje recaudado    object
dtype: object

2018
valor previsto          object
valor registrado        object
valor recaudado         object
porcentaje recaudado    object
dtype: object

2019
valor previsto          object
valor registrado        object
valor recaudado         object
porcentaje recaudado    object
dtype:

In [154]:
# Para convertirlo a float primero tenemos que sustituir la coma por un punto
columnas = ["valor previsto", "valor registrado", "valor recaudado", "porcentaje recaudado"]

for anio in lista_anios:
    for columna in columnas: 
        dic_df[anio][columna]=dic_df[anio][columna].str.replace(",", ".")

In [155]:
dic_df[2021][["valor previsto", "valor registrado", "valor recaudado", "porcentaje recaudado"]].dtypes

valor previsto          object
valor registrado        object
valor recaudado         object
porcentaje recaudado    object
dtype: object

In [156]:
# Convertimos las columnas a tipo float
columnas = ["valor previsto", "valor registrado", "valor recaudado", "porcentaje recaudado"]
for anio in lista_anios:
    for column in columnas:
        dic_df[anio][column]=dic_df[anio][column].astype(float)

In [157]:
for anio in lista_anios:
    print(dic_df[anio][["valor previsto", "valor registrado", "valor recaudado", "porcentaje recaudado"]].dtypes)

valor previsto          float64
valor registrado        float64
valor recaudado         float64
porcentaje recaudado    float64
dtype: object
valor previsto          float64
valor registrado        float64
valor recaudado         float64
porcentaje recaudado    float64
dtype: object
valor previsto          float64
valor registrado        float64
valor recaudado         float64
porcentaje recaudado    float64
dtype: object
valor previsto          float64
valor registrado        float64
valor recaudado         float64
porcentaje recaudado    float64
dtype: object
valor previsto          float64
valor registrado        float64
valor recaudado         float64
porcentaje recaudado    float64
dtype: object
valor previsto          float64
valor registrado        float64
valor recaudado         float64
porcentaje recaudado    float64
dtype: object
valor previsto          float64
valor registrado        float64
valor recaudado         float64
porcentaje recaudado    float64
dtype: object
valor 

Todas las columnas numéricas de las tablas han sido satisfactoriamente convertidas a float.

Explorando estos datos para hacer la conversión me he fijado en que algunos valores estaban a 0 de forma recurrente, indaguemos un poco más en esto.

In [158]:
columnas = ["valor previsto", "valor registrado", "valor recaudado"]
for anio in lista_anios:
    print(f"\n{anio}")
    for columna in columnas:
        print(columna)
        print(f"  {(dic_df[anio][columna].value_counts()/dic_df[anio].shape[0]*100)[0]}")


2013
valor previsto
  34.37083148065807
valor registrado
  94.086260560249
valor recaudado
  15.073365940417963

2014
valor previsto
  35.95431583571272
valor registrado
  94.57500549088513
valor recaudado
  15.506259609048978

2015
valor previsto
  37.453017908467835
valor registrado
  95.06964404156534
valor recaudado
  14.8352863143931

2016
valor previsto
  94.06784453023394
valor registrado
  96.59492219829026
valor recaudado
  1.4845810222430127

2017
valor previsto
  94.0576126502134
valor registrado
  95.08974742622547
valor recaudado
  1.4132791541324765

2018
valor previsto
  93.96874856275583
valor registrado
  97.35834521455182
valor recaudado
  1.4930092443545049

2019
valor previsto
  94.00151559707739
valor registrado
  96.72676272988441
valor recaudado
  1.68695003053815

2020
valor previsto
  93.663416416107
valor registrado
  97.36771854890831
valor recaudado
  2.6147188580099474

2021
valor previsto
  93.6913509617885
valor registrado
  98.05487655375836
valor recau

Estas columnas son de vital importancia ya que reflejan cuando dinero se preve obtener, cuanto se ha regitrado y cuanto se ha recaudado, por lo que ver tantos datos a 0 indica un grave problema burocrático a la hora de llevar las cuentas. Estos valores a 0 se gestionarán en el próximo notebook.

________________

Procedemos a unir las tablas y asi trabajar sobre una única tabla. Los nombres de las columnas y su formato es homogéneo por lo que podemos proceder a la concatenación, método seleccionado ya que cada tabla tiene las mismas columnas y la única opción con sentido es unir las tablas una debajo de otra.

In [159]:
for anio in lista_anios:
    print(dic_df[anio].dtypes)

id organizacion superior           float64
organizacion superior               object
id organizacion                    float64
organizacion                        object
id unidad gestora                  float64
unidad gestora                      object
categoria economica                 object
origen ingreso                      object
tipo ingreso                        object
detalles                            object
valor previsto                     float64
valor registrado                   float64
valor recaudado                    float64
porcentaje recaudado               float64
fecha recaudacion           datetime64[ns]
anio recaudacion                   float64
dtype: object
id organizacion superior           float64
organizacion superior               object
id organizacion                    float64
organizacion                        object
id unidad gestora                  float64
unidad gestora                      object
categoria economica                 obje

In [160]:
lista_df=[]
for anio in lista_anios:
    lista_df.append(dic_df[anio])

In [161]:
suma = 0
for anio in lista_anios:
    suma += dic_df[anio].shape[0]
suma

1026299

In [162]:
df_concat = pd.concat(lista_df, axis=0, ignore_index = True)
df_concat.shape


(1026299, 16)

Comprobamos satisfactoriamente que la concatenación se ha realizado comprobando que el número de filas totales es la suma de las filas de cada tabla. Eliminamos filas duplicadas, pasamos la fecha a formato date, elminamoscolumnas innecesarias y guardamos a nuestro pequeño monstruo en un nuevo csv.

In [163]:
df_concat.drop_duplicates(inplace=True)

In [164]:
df_concat.shape

(1026214, 16)

In [165]:
df_concat.dtypes

id organizacion superior    float64
organizacion superior        object
id organizacion             float64
organizacion                 object
id unidad gestora           float64
unidad gestora               object
categoria economica          object
origen ingreso               object
tipo ingreso                 object
detalles                     object
valor previsto              float64
valor registrado            float64
valor recaudado             float64
porcentaje recaudado        float64
fecha recaudacion            object
anio recaudacion            float64
dtype: object

In [166]:
df_concat["fecha recaudacion"]= pd.to_datetime(df_concat["fecha recaudacion"])

In [167]:
df_concat.dtypes

id organizacion superior           float64
organizacion superior               object
id organizacion                    float64
organizacion                        object
id unidad gestora                  float64
unidad gestora                      object
categoria economica                 object
origen ingreso                      object
tipo ingreso                        object
detalles                            object
valor previsto                     float64
valor registrado                   float64
valor recaudado                    float64
porcentaje recaudado               float64
fecha recaudacion           datetime64[ns]
anio recaudacion                   float64
dtype: object

In [168]:
df_concat.drop("detalles", axis = 1, inplace = True) #Columnna inecesaria por lo que la quitamos

In [169]:
df_concat.columns

Index(['id organizacion superior', 'organizacion superior', 'id organizacion',
       'organizacion', 'id unidad gestora', 'unidad gestora',
       'categoria economica', 'origen ingreso', 'tipo ingreso',
       'valor previsto', 'valor registrado', 'valor recaudado',
       'porcentaje recaudado', 'fecha recaudacion', 'anio recaudacion'],
      dtype='object')

In [170]:
df_concat.to_csv("../datos/tablas_concatenadas.csv")

In [171]:
df_concat.isnull().sum()

id organizacion superior     30359
organizacion superior       359187
id organizacion              25117
organizacion                 34886
id unidad gestora            33581
unidad gestora               19481
categoria economica          18978
origen ingreso               38417
tipo ingreso                 31927
valor previsto               51315
valor registrado             26419
valor recaudado              39442
porcentaje recaudado         24134
fecha recaudacion            23747
anio recaudacion                 0
dtype: int64